In [1]:
from collections import namedtuple
import math

class P3(namedtuple('P3', ['x', 'y', 'z'])):
    __slots__ = ()
    def __new__(self, x, y, z):
        return super(P3, self).__new__(self, float(x), float(y), float(z))
    def __repr__(self):
        return "P3(%s, %s, %s)" % (self.x, self.y, self.z)
    def __add__(self, a):
        return P3(self.x + a.x, self.y + a.y, self.z + a.z)
    def __sub__(self, a):
        return P3(self.x - a.x, self.y - a.y, self.z - a.z)
    def __mul__(self, a):
        return P3(self.x*a, self.y*a, self.z*a)
    def __neg__(self):
        return P3(-self.x, -self.y, -self.z)
    def __rmul__(self, a):
        raise TypeError
    def Lensq(self):
        return self.x*self.x + self.y*self.y + self.z*self.z
    def Len(self):
        return math.sqrt(self.Lensq())
    def LenLZ(self):
        return math.sqrt(self.x*self.x + self.y*self.y)
        
        
    def assertlen1(self):
        assert abs(self.Len() - 1.0) < 0.0001
        return True
        
    @staticmethod
    def Dot(a, b):
        return a.x*b.x + a.y*b.y + a.z*b.z

    @staticmethod
    def Cross(a, b):
        return P3(a.y*b.z - b.y*a.z, -a.x*b.z + b.x*a.z, a.x*b.y - b.x*a.y)


In [23]:
# ab is the triangle edge axis, d is crossing point on axis coming from c
# e is point of triangle to right of axis
Ga = P3(0,0,0)
Gb = P3(0,1,0)
lam = 0.31
Gc = P3(-1,0.3,0)
Ge = P3(1,0.5,0)


In [31]:
# axis is a b
# incoming at d = Along(lam, a, b)
# incoming from c to d 
# incoming (d - c).(b - a) = |d-c||b-a|cos(theta) = |d-c|I
# I = |b-a|cos(theta) = (d - c).(b - a)/|d-c|

# triangle on right hand side is to e
# outgoing to x where x = Along(q, a, e) or x = Along(q, b, e)
# Solve (x - d).(b - a) = |x-d||b-a|cos(theta) = |x-d|I

# Set d = 0, v = b - a
# x.v = |x|I  where x = a + (e - a)*q = a + f*q
# x.x I^2 = (x.v)^2

# x.x I^2 = (a + f*q)^2 I^2 = q^2 f^2 I^2 + 2q a.f I^2 + a^2 I^2
#   =
# (x.v)^2 = (a.v + f.v*q)^2 = q^2 (f.v)^2 + 2q (a.v)(f.v) + (a.v)^2
# q^2 ((f.v)^2 - f^2 I^2) + 2q ((a.v)(f.v) - a.f I^2) + (a.v)^2 - a^2 I^2

def Square(X):
    return X*X
def TOL_ZERO(X):
    if not (abs(X) < 0.0001):
        print("TOL_ZERO fail", X)

def GeoCrossAxisE(a, Vae, Vab, Isq, Isgn):
    fdv = P3.Dot(Vae, Vab)
    adv = P3.Dot(a, Vab)
    adf = P3.Dot(a, Vae)
    qA = Square(fdv) - Vae.Lensq()*Isq
    qB2 = adv*fdv - adf*Isq
    qC = a.Lensq() * (1 - Isq)
    
    qdq = Square(qB2) - qA*qC
    if qdq < 0.0:
        return -1.0
    qs = math.sqrt(qdq) / qA
    qm = -qB2 / qA
    q = qm + qs*Isgn
    # q = qs +- qm,  x = a + Vae*q,  Dot(x, Vab) same sign as Dot(Vcd, Vab)
    if abs(q) < 100:
        TOL_ZERO(qA*Square(q) + qB2*2*q + qC)
    return q

def GeoCrossAxis(Ga, Gb, Gc, lam, Ge):
    Vab = Gb - Ga
    Gd = Ga + Vab*lam
    Vcd = Gd - Gc
    cdDab = P3.Dot(Vcd, Vab)
    Isq = Square(cdDab) / Vcd.Lensq()
    Isgn = -1 if cdDab < 0 else 1
    qVae = GeoCrossAxisE(Ga - Gd, Ge - Ga, Vab, Isq, Isgn)
    qVbe = GeoCrossAxisE(Gb - Gd, Ge - Gb, -Vab, Isq, -Isgn)
    bAEcrossing = (abs(qVae - 0.5) < abs(qVbe - 0.5))
    q = qVae if bAEcrossing else qVbe
    Gx = (Ga + (Ge - Ga)*q) if bAEcrossing else (Gb + (Ge - Gb)*q)
    Dx = Gx - Gd
    TOL_ZERO(P3.Dot(Vcd, Vab)/Vcd.Len() - P3.Dot(Dx, Vab)/Dx.Len())
    return bAEcrossing, q
    
GeoCrossAxis(Ga, Gb, Gc, lam, Ge)

#        assert abs(qc + (qm + qs)*(2*qb2 + (qm + qs)*self.vpsq)) < 0.002
#        if qm + qs <= 0.0:
#            return
#        laml = qm - qs


(True, 0.6326530612244927)

In [91]:
x0, x1
# Solve (x - d).(b - a) = |x-d||b-a|cos(theta) = |x-d|I
math.sqrt(Isq), P3.Dot(x0, Vab)/x0.Len(), P3.Dot(x1, Vab)/x1.Len()

(0.009999500037496884, 0.00999950003749914, -0.009999500037499243)

In [83]:
l, qs, qm, qdq, q0, q1

(0.6, 0.0, 0.6, 0.0, 0.6, 0.6)

In [75]:
qA, qB2, qC

(0.25, -0.15, 0.09)

In [72]:
adv

-0.3